In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from scipy import stats
import chart_studio.plotly as py
import plotly.express as px
import cufflinks as cf
import plotly.graph_objects as go
import seaborn as sns

In [2]:
name_df = pd.read_csv("https://datasets.imdbws.com/name.basics.tsv.gz", sep="\t")
name_df.head(1)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0031983,tt0050419,tt0053137"


In [3]:
akas_df = pd.read_csv("https://datasets.imdbws.com/title.akas.tsv.gz", sep="\t", dtype={"titleId": "string", "ordering": str, "title ": "string", "region ": "string", 
"language ": "string", "types ": str, "attributes ": str, "isOriginalTitle": str})
akas_df.head(1)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0


In [3]:
basics_df = pd.read_csv("https://datasets.imdbws.com/title.basics.tsv.gz", sep="\t", low_memory=False)
basics_df.head(1)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"


In [ ]:
crew_df = pd.read_csv("https://datasets.imdbws.com/title.crew.tsv.gz", sep="\t", low_memory=False)
crew_df.head(1)

In [ ]:
episode_df = pd.read_csv("https://datasets.imdbws.com/title.episode.tsv.gz", sep="\t", low_memory=False)
episode_df.head(1)

In [4]:
principals_df = pd.read_csv("https://datasets.imdbws.com/title.principals.tsv.gz", sep="\t")
principals_df.head(1)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"


In [ ]:
ratings_df = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz", sep="\t", low_memory=False)
ratings_df.head(1)

# <span style="color:orange">Quels sont les pays qui produisent le plus de films ?</span> 

In [5]:
# Quels sont les pays qui produisent le plus de films ?
# étape 1 : juste un value_counts sur les régions dans la Database akas en prenant le top 15
movie_By_Region2 = akas_df["region"].value_counts().reset_index().head(15)
# je retire les régions \N
movie_By_Region3 = movie_By_Region2[movie_By_Region2['index'] != '\\N']

fig = px.bar(movie_By_Region3, x="index", y ='region', color = 'index', text ='region',
    title = 'Quels sont les pays qui produisent le plus de films ?',
    labels = {'region': 'Nombre de films', 'index': 'Pays'},
    width=800, height=600)
    
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8)

fig.update_layout(showlegend=False, title_x=0.5, yaxis={'visible': False})

NameError: name 'akas_df' is not defined

# <span style="color:orange">Quels sont les acteurs les plus présents ?</span> 

In [6]:
# Quels sont les acteurs les plus présents ?
# étape 1 : nettoyage de la Database

principals_df2 = principals_df[principals_df['category'].str.contains('actor|actress')]
principals_df3 = principals_df2[principals_df2['characters'] != '\\N']
principals_df4 = principals_df3[principals_df3['characters'] != 'Narrator']
principals_df5 = principals_df4[principals_df4['characters'] != 'Various']
principals_df6 = principals_df5[principals_df5['characters'] != 'Various Characters']
principals_df7 = principals_df6[principals_df6['characters'] != 'Additional Voices']

In [7]:
# Quels sont les acteurs les plus présents ?
# étape 2 : set index puis merge
# set index on tconst for basics and principal table
basics_df2 = basics_df.set_index('tconst')
principals_df8 = principals_df7.set_index('tconst')
# Merge de la table principals et basics afin de filtrer uniquement sur les movies et les shorts
merge_basics_principals = pd.merge(basics_df2, principals_df8, how='inner', left_index=True, right_index=True)

In [8]:
# Quels sont les acteurs les plus présents ?
# étape 3 : Nettoyage pour garder uniquement les titleType movie et short
merge_basics_principals2 = merge_basics_principals[(merge_basics_principals['titleType'] == 'movie') | (merge_basics_principals['titleType'] == 'short')]

In [9]:
# Quels sont les acteurs les plus présents ?
# étape 4 : set index puis merge
# set index on nconst for Database name and Database merge_basics_principals2 to prepare a merge
name_df2 = name_df.set_index('nconst')
merge_basics_principals3 = merge_basics_principals2.set_index('nconst')
# merge to link first name with nconst
merge_basics_principals4 = pd.merge(name_df2, merge_basics_principals3, how='inner', left_index=True, right_index=True)

In [10]:
# Quels sont les acteurs les plus présents ?
# étape 5 : Nettoyage pour retirer les oeuvres très courtes comme les cartoons de quelques minutes pour lesquelles les acteurs font seulement les voix
merge_basics_principals5 = merge_basics_principals4[merge_basics_principals4['runtimeMinutes'] != '\\N']
merge_basics_principals6 = merge_basics_principals5.astype({"runtimeMinutes": int})
merge_basics_principals7 = merge_basics_principals6[merge_basics_principals6['runtimeMinutes'] > 20]

In [11]:
# Quels sont les acteurs les plus présents ?
# étape 6 : Value_counts pour trouver les acteurs les plus présents
merge_basics_principals8 = merge_basics_principals7['primaryName'].value_counts()
merge_basics_principals9 = merge_basics_principals8.head(20)

In [12]:
# Quels sont les acteurs les plus présents ?
# étape 7 : visualisation
fig = px.bar(merge_basics_principals9.reset_index(), x="primaryName", y ='index', color = 'index',
    title = 'Quels sont les acteurs les plus présents ?',
    labels = {'primaryName': 'Nombre de films', 'index': 'Acteurs'},
    width=800, height=600)

fig.update_layout(showlegend=False, title_x=0.5)

# <span style="color:orange">Quels sont les acteurs les plus présents ? A quelle période ?</span> 

In [13]:
# Quels sont les acteurs les plus présents ? A quelle période ?
# étape 1 : Nettoyage de la base de données pour retirer les titres qui n'ont pas de startYear
merge_basics_principals8 = merge_basics_principals7[merge_basics_principals7['startYear'] != "\\N"]

In [14]:
# Quels sont les acteurs les plus présents ? A quelle période ?
# étape 1 : Nettoyage de la base de données pour retirer les titres qui n'ont pas de startYear
merge_basics_principals_test = merge_basics_principals8[["primaryName", "startYear"]]
merge_basics_principals_test2 = merge_basics_principals_test.reset_index(drop=True)

In [15]:
def find_decade(year):
    decade1 = (year // 10 * 10)
    decade2 = (year // 10 * 10) + 10
    return str(decade1) + " - " + str(decade2)

print(find_decade(1934))


1930 - 1940


In [123]:
merge_basics_principals_test4 = merge_basics_principals_test2.astype({"startYear": int})
merge_basics_principals_test4["startYear"] = merge_basics_principals_test4["startYear"].apply(find_decade)

In [22]:
#merge_basics_principals_test4.head(60)
merge_basics_principals_test4["primaryName"].value_counts()

Mohanlal                  235
Mammootty                 224
Eric Roberts              222
Cüneyt Arkin              219
Raymond Hatton            202
                         ... 
Christopher Brändström      1
Steve Brobston              1
Gwennaël Breës              1
Caroline Bielskis           1
Chinmay Mishra              1
Name: primaryName, Length: 491624, dtype: int64

In [18]:
fig = px.bar(merge_basics_principals_test4, x="primaryName", y ='startYear', color = 'startYear',
    title = 'Quels sont les acteurs les plus présents ?',
    labels = {'primaryName': 'Nombre de films', 'startYear': 'Acteurs'},
    width=800, height=600)

fig.update_layout(showlegend=False, title_x=0.5)

KeyboardInterrupt: 

In [124]:
merge_basics_principals_test4.reset_index(inplace=True)


In [125]:
df_grouped_notreset  = pd.DataFrame({'count' : merge_basics_principals_test4.groupby(['startYear', 'primaryName'] ).size()})

In [126]:
df_grouped_notreset_sorted=df_grouped_notreset.sort_values(['startYear', 'count'], ascending=False)

In [127]:
groupedDf = merge_basics_principals_test4.groupby(['startYear', 'primaryName'] ).size()

In [128]:
df_final  = pd.DataFrame({'count' : groupedDf.groupby(level='startYear').nlargest(5).reset_index(level=0, drop=True)})

In [149]:
df_final2 = df_final.reset_index
print (df_final2)

<bound method DataFrame.reset_index of                                    count
startYear   primaryName                 
1890 - 1900 Blanche Bayliss            1
            Chauncey Depew             1
            William Courtenay          1
1900 - 1910 Adelaide Fitz-Allen        1
            Alexandre Arquillière      1
...                                  ...
2020 - 2030 Eric Roberts              30
            Ross K. Foad              24
            Simon Hill                24
            Cascade Nerida            15
            Shawn C. Phillips         15

[68 rows x 1 columns]>


In [145]:
fig = px.bar(df_final, x="primaryName", y ='count', color = 'count',
    title = 'Quels sont les acteurs les plus présents ?',
    labels = {'primaryName': 'Nombre de films', 'count': 'Acteurs'},
    width=800, height=600)

fig.update_layout(showlegend=False, title_x=0.5)

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['count'] but received: primaryName

In [147]:
df_final2.info()

AttributeError: 'function' object has no attribute 'info'

In [38]:
#merge_basics_principals_test5 = merge_basics_principals_test4.pivot(columns="startYear")
#merge_basics_principals_test6 = merge_basics_principals_test5.apply(pd.Series.value_counts)
#merge_basics_principals_test7 = merge_basics_principals_test6.fillna(0)




                            primaryName                                      \
startYear                   1890 - 1900 1900 - 1910 1910 - 1920 1920 - 1930   
''Knife'' Sotelo                    0.0         0.0         0.0         0.0   
'Ace Primo' Niko Warren             0.0         0.0         0.0         0.0   
'Baby' Carmen De Rue                0.0         0.0         5.0         0.0   
'Big' Bill Wilson                   0.0         0.0         0.0         1.0   
'Big' LeRoy Mobley                  0.0         0.0         0.0         0.0   
...                                 ...         ...         ...         ...   
Þórhallur Þórhallsson               0.0         0.0         0.0         0.0   
Þórhildur Ýr Arnardóttir            0.0         0.0         0.0         0.0   
Þórir Waagfjörð                     0.0         0.0         0.0         0.0   
Þórunn Arna Kristjánsdóttir         0.0         0.0         0.0         0.0   
Þórður Aðalbjörnsson                0.0         0.0 

In [39]:
#merge_basics_principals_test8 = merge_basics_principals_test7.transform(np.sort)
#merge_basics_principals_test8.tail(60)

                            primaryName                                      \
startYear                   1890 - 1900 1900 - 1910 1910 - 1920 1920 - 1930   
''Knife'' Sotelo                    0.0         0.0         0.0         0.0   
'Ace Primo' Niko Warren             0.0         0.0         0.0         0.0   
'Baby' Carmen De Rue                0.0         0.0         0.0         0.0   
'Big' Bill Wilson                   0.0         0.0         0.0         0.0   
'Big' LeRoy Mobley                  0.0         0.0         0.0         0.0   
...                                 ...         ...         ...         ...   
Þórhallur Þórhallsson               0.0         1.0        40.0        55.0   
Þórhildur Ýr Arnardóttir            0.0         1.0        40.0        58.0   
Þórir Waagfjörð                     1.0         1.0        40.0        58.0   
Þórunn Arna Kristjánsdóttir         1.0         1.0        41.0        63.0   
Þórður Aðalbjörnsson                1.0         1.0 

In [1]:
#quelques tests

#name_df6.loc[name_df6['primaryName'] == "Nikita"]

#name_df6.loc[name_df6['primaryName'] == "Brad Pitt"]

#principals_df.loc[(principals_df['nconst'] == "nm0000093") & (principals_df['category'] == "actor")]

#principals_df[principals_df['category'].str.contains('actor|actress')]
#principals_df['nconst'].value_counts()

#name_df.loc[name_df['nconst'] == "nm10120013"]

#principals_df2 = principals_df[principals_df['category'].str.contains('actor|actress')]

#principals_df2['nconst'].value_counts()

#principals_df.loc[principals_df['nconst'] == "nm0000093"]

# name_df6['numberOfKnownForTitles'] = name_df6['knownForTitles'].str.count("tt")
# name_df6.sort_values(by=['numberOfKnownForTitles'], ascending=False).head(15)

# name_df2 = name_df.dropna(axis=0)
# name_df3 = name_df2[name_df2['primaryProfession'] != '\\N']
# name_df4 = name_df3[name_df3['birthYear'] != '\\N']
# name_df5 = name_df4[name_df4['knownForTitles'] != '\\N']
# name_df6 = name_df5[name_df5['primaryProfession'].str.contains('actor|actress')]

#akas_df2 = akas_df[(akas_df['region'] == 'FR') & (akas_df['title'] == 'Star Wars : Épisode VII - Le Réveil de la Force')]
#akas_df2 = akas_df[akas_df['region'] == 'FR']
#akas_df2 = akas_df[(akas_df['title'] == 'Mr. & Mrs. Smith') & (akas_df['region'] == 'FR')]
#akas_df2 = akas_df[akas_df['titleId'] == 'tt0000001']
#akas_df2 = akas_df[akas_df['title'] == 'Mr. & Mrs. Smith']